In [ ]:
import numpy as np
import pandas as pd

In [ ]:
book = pd.read_csv('./Dataset/Books.csv')
rating = pd.read_csv('./Dataset/Ratings.csv')
user = pd.read_csv('./Dataset/Users.csv')

### Popularity Based

In [ ]:
rateWithBook = rating.merge(book,on='ISBN')
numRateDf = rateWithBook.groupby('Book-Title').count()['Book-Rating'].reset_index()
numRateDf.rename(columns={'Book-Rating':'num_rating'}, inplace=True)

In [ ]:
avgRateDf = rateWithBook.groupby('Book-Title')['Book-Rating'].mean().reset_index()
avgRateDf.rename(columns={'Book-Rating':'avg_rating'}, inplace=True)

In [ ]:
popularityDf = numRateDf.merge(avgRateDf,on='Book-Title')
popularDF = popularityDf[popularityDf['num_rating']>=250].sort_values('avg_rating',ascending=False).head(50)

In [ ]:
popularDf = popularDF.merge(book, on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_rating', 'avg_rating']]
popularDf

In [ ]:
import pickle
pickle.dump(popularDf, open('popular.pkl', 'wb'))

### Collaborative Based

In [ ]:
x = rateWithBook.groupby('User-ID').count()['Book-Rating'] > 200
imp_users = x[x].index

In [ ]:
filtered_rating = rateWithBook[rateWithBook['User-ID'].isin(imp_users)]

In [ ]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index

In [ ]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]
pt = final_ratings.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating')
pt.fillna(0, inplace = True)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_score = cosine_similarity(pt)

In [ ]:
def recommend(book_name):
    #index fetch
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_score[index])), key=lambda x:x[1], reverse=True)[1:6]

    data = []
    for i in similar_items:
        item = []
        temp_df = book[book['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))

        data.append(item)

    return data

In [ ]:
pickle.dump(pt, open('pt.pkl', 'wb'))
pickle.dump(book, open('books.pkl', 'wb'))
pickle.dump(similarity_score, open('similarity_score.pkl', 'wb'))

In [ ]:
recommend('Animal Farm')